In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
import os
os.listdir("input/")

['test.csv', 'train.csv', 'sample_submission.csv']

In [3]:
train = pd.read_csv("input/train.csv", dtype={"fullVisitorId": "str"})
test = pd.read_csv("input/test.csv", dtype={"fullVisitorId": "str"})

In [4]:
train.head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [6]:
type(train.fullVisitorId[0])

str

In [7]:
columns_json = ['device', 'geoNetwork', 'totals', 'trafficSource']
dt_test = train.head(5)
dt_test

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [16]:
dt_test.shape

(5, 12)

In [8]:
import json
from pandas.io.json import json_normalize

In [11]:
def load_data(path, nrows=None):
    columns_json = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(
                        path, 
                        converters={column: json.loads for column in columns_json}, 
                        dtype={'fullVisitorId':'str'})
    for column in columns_json:
        df_columns_json = json_normalize(df[column])
        df_columns_json.columns = [f"{column}.{subcolumn}" for subcolumn in df_columns_json.columns]
        df = df.drop(column, axis=1).merge(df_columns_json, left_index=True, right_index=True)
    return df

In [12]:
%%time
train = load_data("input/train.csv")
test = load_data("input/test.csv")

CPU times: user 6min 33s, sys: 4min 59s, total: 11min 32s
Wall time: 13min 9s


In [14]:
train.to_csv("input/train_extend.csv", index=False)
test.to_csv("input/test_extend.csv", index=False)

In [19]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [16]:
test.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,Chrome,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,Safari,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,Safari,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google


In [17]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (903653, 55)
test shape:  (804684, 53)


trainset have 55 column while having 53 in testset. We knew that totals.transactionRevenue in training set is our target. We should find a column different between two set and drop it in training set

In [34]:
target = train['totals.transactionRevenue']
col_drop = set(train.columns).difference(set(test.columns))
print(col_drop)

train = train.drop(col_drop, axis=1)

{'totals.transactionRevenue', 'trafficSource.campaignCode'}


In [36]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)
print("target shape: ", target.shape)

train shape:  (903653, 53)
test shape:  (804684, 53)
target shape:  (903653,)


## Try to predict log(transactionRevenue) by training model using lightgbm

In [2]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

/Users/nguyenkhacbaoanh/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:45: FutureWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", FutureWarning)


In [18]:
train = pd.read_csv("input/train_extend.csv", dtype={'fullVisitorId':'str'})
test = pd.read_csv("input/test_extend.csv", dtype={'fullVisitorId':'str'})

/Users/nguyenkhacbaoanh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
target = train['totals.transactionRevenue']
col_drop = set(train.columns).difference(set(test.columns))
print(col_drop)

train = train.drop(col_drop, axis=1)

{'trafficSource.campaignCode', 'totals.transactionRevenue'}


In [21]:
target = np.log1p(target.fillna(0).astype('float64'))

drop all of constants columns in training set

In [22]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False) == 1]
const_cols

['socialEngagementType',
 'device.browserSize',
 'device.browserVersion',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [23]:
train = train.drop(const_cols, axis=1)
test = test.drop(const_cols, axis=1)

In [60]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (903653, 34)
test shape:  (804684, 34)


### Checking missing value

In [57]:
def check_missing_dt(train, test):
    missing_train = train.isnull().sum().values # array of all number of missing value per column
    missing_test = test.isnull().sum().values
    missing_train_per = missing_train/train.shape[0] # calculate percentage of missing value on total samples
    missing_test_per = missing_test/test.shape[0]
    return pd.DataFrame({"missing value's percentage of training": missing_train_per, 
                        "missing value's percentage of testing": missing_test_per}, 
                        index=train.columns).sort_values(
                                by=["missing value's percentage of training", "missing value's percentage of testing"], 
                                ascending=False)

In [58]:
missing_data = check_missing_dt(train, test)
missing_data

,missing value's percentage of training,missing value's percentage of testing
trafficSource.adContent,0.987887,0.933153
trafficSource.adwordsClickInfo.adNetworkType,0.976252,0.933124
trafficSource.adwordsClickInfo.isVideoAd,0.976252,0.933124
trafficSource.adwordsClickInfo.page,0.976252,0.933124
trafficSource.adwordsClickInfo.slot,0.976252,0.933124
trafficSource.adwordsClickInfo.gclId,0.976140,0.933064
trafficSource.isTrueDirect,0.696781,0.676254
trafficSource.referralPath,0.633774,0.707558
trafficSource.keyword,0.556551,0.485945
totals.bounces,0.501324,0.476878


In [27]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,NaN,NaN,NaN,NaN,(not set),NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


Just the way that i struggle to seperate categorical columns and numerical columns

In [59]:
cat_cols = []
num_cols = []
for c in train.columns:
    try:
        train[c].astype(np.float64)
        num_cols.append(str(c))
    except ValueError:
        cat_cols.append(str(c))
print("categorical columns: ",cat_cols)
print()
print("numerical columns:",num_cols)

categorical columns:  ['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source']

numerical columns: ['date', 'fullVisitorId', 'sessionId', 'visitId', 'visitNumber', 'visitStartTime', 'device.isMobile', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.page', 'trafficSource.isTrueDirect']


    the function to plot the categorical columns

In [83]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [84]:
device = train['device.browser'].value_counts()[:6]
print(device.values)
data = [go.Bar(
            x= device.index,
            y= device.values
)]
iplot(data, filename="Device Browser")

[620364 182245  37069  19375  10205   7865]


In [69]:
list(device.index)

['Chrome', 'Safari', 'Firefox', 'Internet Explorer', 'Edge', 'Android Webview']

In [ ]:
def show_cat_cols(data):
    

In [71]:
X_train, X_val, y_train, y_val = train_test_split(train.iloc[:9000,:],target[:9000], test_size=0.2, random_state=42)

In [72]:
data_train = lgb.Dataset(X_train, label=y_train)
data_val = lgb.Dataset(X_val, label=y_val)

In [73]:
param = {'boosting_type': 'gbdt',
         'application':'regression',
         'metric': 'mean_squared_error',
         'learning_rate':0.05,
         'num_leaves':100,
         'min_data_in_leaf':150,
         'max_depth':-1,
         'bagging_fraction':0.7,
         'feature_fraction':0.7,
         'max_bin':200,
         'random_state': 42}

In [74]:
model = lgb.train(param, data_train, 1000)

ValueError: DataFrame.dtypes for data must be int, float or bool. Did not expect the data types in fields channelGrouping, fullVisitorId, sessionId, socialEngagementType, device.browser, device.browserSize, device.browserVersion, device.deviceCategory, device.flashVersion, device.language, device.mobileDeviceBranding, device.mobileDeviceInfo, device.mobileDeviceMarketingName, device.mobileDeviceModel, device.mobileInputSelector, device.operatingSystem, device.operatingSystemVersion, device.screenColors, device.screenResolution, geoNetwork.city, geoNetwork.cityId, geoNetwork.continent, geoNetwork.country, geoNetwork.latitude, geoNetwork.longitude, geoNetwork.metro, geoNetwork.networkDomain, geoNetwork.networkLocation, geoNetwork.region, geoNetwork.subContinent, totals.bounces, totals.hits, totals.newVisits, totals.pageviews, totals.visits, trafficSource.adContent, trafficSource.adwordsClickInfo.adNetworkType, trafficSource.adwordsClickInfo.criteriaParameters, trafficSource.adwordsClickI

In [85]:
train.to_csv("input/train_work.csv", index=False)
test.to_csv("input/test_work.csv", index=False)
target.to_csv("input/target_work.csv", index=False)

In [ ]:
train = pd.read_csv("input/train_work.csv")